In [37]:
from gurobipy import Model, GRB, quicksum

In [38]:
# Create a new model
m = Model("electric_charging_stations")

In [39]:
# Set of companies and sites
companies = [1, 2]
sites = list(range(1, 31))

In [40]:
# Fixed cost for building a station
fixed_cost = 2500000

# Variable cost per charging unit
variable_cost = 150000

In [41]:
# Decision variables
x = m.addVars(companies, sites, vtype=GRB.BINARY, name="x")
y = m.addVars(companies, sites, vtype=GRB.INTEGER, name="y")

In [42]:
# # Objective: Minimize total cost
# m.setObjective(
#     quicksum(2500000 * x[i, n] + 150000 * y[i, n] for i in companies for n in sites), 
#     GRB.MINIMIZE
# )

In [43]:
# Objective: Minimize total cost
m.setObjective(
    quicksum(fixed_cost * (x[1, n] + x[2, n]) + variable_cost * (y[1, n] + y[2, n])
             for n in sites), 
    GRB.MINIMIZE
)

In [44]:
# Constraints

# a) At least 14 stations must be built
m.addConstr(quicksum(x[i, n] for i in companies for n in sites) >= 14)

# b) No more than 18 units at any station
for i in companies:
    for n in sites:
        m.addConstr(y[i, n] <= 18 * x[i, n])

# e) Company 2 must build exactly 4 stations among specific sites
m.addConstr(quicksum(x[2, n] for n in [3, 11, 17, 19, 23]) == 4)

# f) At least one station at sites 12, 15, and 18
m.addConstr(quicksum(x[i, n] for i in companies for n in [12, 15, 18]) >= 1)

# Rest of the constraints based on the problem statement

# Constraint to ensure only one company builds at a site
for n in sites:
    m.addConstr(quicksum(x[i, n] for i in companies) <= 1)

# If a station is built at site 2, then a station must also be built at site 30
for i in companies:
    m.addConstr(x[i, 2] <= x[i, 30])

# Company 1 cannot build at both sites 8 and 21
m.addConstr(x[1, 8] + x[1, 21] <= 1)

# If company 1 builds a station at site 1, they must also build at sites 26, 27, 28
m.addConstr(x[1, 1] <= x[1, 26])
m.addConstr(x[1, 1] <= x[1, 27])
m.addConstr(x[1, 1] <= x[1, 28])

# The number of units at station 14 must equal the number at station 22
m.addConstr(quicksum(y[i, 14] for i in companies) == quicksum(y[i, 22] for i in companies))


<gurobi.Constr *Awaiting Model Update*>

In [45]:
# Solve model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 120 columns and 267 nonzeros
Model fingerprint: 0xee00405d
Variable types: 0 continuous, 120 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+05, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 3.500000e+07
Presolve removed 100 rows and 120 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3.5e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.500000000000e+07, best bound 3.500000000000e+07, gap 0.0000%


In [46]:
# Print solution
if m.status == GRB.OPTIMAL:
    print('Optimal cost: %g' % m.objVal)
    for i in companies:
        for n in sites:
            if x[i, n].x > 0.5:  # If station is built
                print('Company %d builds a station at site %d with %d units.' % (i, n, y[i, n].x))
else:
    print('No optimal solution found.')

Optimal cost: 3.5e+07
Company 1 builds a station at site 10 with 0 units.
Company 1 builds a station at site 15 with 0 units.
Company 1 builds a station at site 16 with 0 units.
Company 1 builds a station at site 27 with 0 units.
Company 1 builds a station at site 29 with 0 units.
Company 2 builds a station at site 4 with 0 units.
Company 2 builds a station at site 5 with 0 units.
Company 2 builds a station at site 6 with 0 units.
Company 2 builds a station at site 8 with 0 units.
Company 2 builds a station at site 11 with 0 units.
Company 2 builds a station at site 13 with 0 units.
Company 2 builds a station at site 17 with 0 units.
Company 2 builds a station at site 19 with 0 units.
Company 2 builds a station at site 23 with 0 units.
